<a href="https://colab.research.google.com/github/local-ai-gr/CNROpt/blob/main/CNROpt_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNROpt Model Training Code for the Forecasting Phase**

Importing Facebook Prophet Forecasting Module

In [ ]:
!pip install prophet



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install functions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for functions: filename=functions-0.7.0-py3-none-any.whl size=3037 sha256=e3542bb5906022ae7d22e2850fee99e8b6729729ae03b4121419ba724b5ff105
  Stored in directory: /root/.cache/pip/wheels/92/90/bc/01eb2f163bcfa49c7ad5072fbe9d243038368b9001ec26a9da
Successfully built functions


In [18]:
import pandas as pd
import prophet as ph
import psycopg2
import os
import sys


Connecting to PostGIS Database (host: 34.171.207.247:5432
db: localai) and loading data (all but the test set) to pandas Data Frame

In [27]:
conn = psycopg2.connect(
                   host='34.171.207.247',
                   database='localai',
                   user='postgres',
                   password='pglocalai')

In [23]:
query = "select * from station_data_hour"
cursor = conn.cursor()
cursor.execute(query)
tupples = cursor.fetchall()

column_names = [desc[0] for desc in cursor.description]
for i in column_names:
    print(i)
conn.commit()
conn.close()

df = pd.DataFrame(tupples, columns=column_names)



location_id
location_friendlyname
location_friendlycode
location_address_cityname
location_address_postnumber
location_address_streetname
location_address_housenumber
location_accesstype_title
charging_hour
utilization
connector_count
congested_connectors
energy_consumption


Visualize the results using matplotlib and folium maps

In [25]:
df.head()

,location_id,location_friendlyname,location_friendlycode,location_address_cityname,location_address_postnumber,location_address_streetname,location_address_housenumber,location_accesstype_title,charging_hour,utilization,connector_count,congested_connectors,energy_consumption
0,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 12:00:00+00:00,5.000000,7,0,2.477495
1,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 13:00:00+00:00,14.285714,7,1,7.078557
2,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 14:00:00+00:00,13.095238,7,0,6.488678
3,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-16 07:00:00+00:00,1.428571,7,0,1.778800
4,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-16 08:00:00+00:00,2.142857,7,0,2.668201


In [37]:
df2 = df['utilization'].groupby(df['location_friendlyname']).mean()

In [41]:
df2.sort_values()

location_friendlyname
Vukova GoricaRI            3.310372
Vukova GoricaZG            4.298981
Ljubeščica Zapad           4.910647
Lepenica                   5.009270
Prokljan Jug               5.035951
                            ...    
Zagreb Langić             54.568941
eRoaming Showcase Ebee    56.666667
SamoborHEP                80.000000
Varaždinske Toplice       85.109184
VinodolHEP HE             95.893510
Name: utilization, Length: 257, dtype: float64

In [52]:
df[df['location_friendlyname'] == 'VinodolHEP HE']

,location_id,location_friendlyname,location_friendlycode,location_address_cityname,location_address_postnumber,location_address_streetname,location_address_housenumber,location_accesstype_title,charging_hour,utilization,connector_count,congested_connectors,energy_consumption
430757,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2021-05-10 12:00:00+00:00,31.666667,1,0,2.139833
430758,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2021-05-10 13:00:00+00:00,98.333333,1,0,6.109959
430759,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2021-05-10 14:00:00+00:00,93.333333,1,0,0.391941
430760,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2021-05-10 15:00:00+00:00,100.000000,1,1,0.000000
430761,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2021-05-10 16:00:00+00:00,100.000000,1,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
433397,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2022-10-24 15:00:00+00:00,3.333333,1,0,0.149146
433398,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2022-10-26 07:00:00+00:00,98.333333,1,0,4.425342
433399,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2022-10-26 08:00:00+00:00,45.000000,1,0,2.025156
433400,240498,VinodolHEP HE,HRHEP-51029,Tribalj,51243,Sušik,15,Public access but not visible to public,2022-10-28 13:00:00+00:00,70.000000,1,0,3.128824


In [57]:
dfv = df[df['location_friendlyname'] == 'VinodolHEP HE'][['location_friendlyname', 'charging_hour', 'utilization']]

In [58]:
dfv.sort_values('charging_hour')

,location_friendlyname,charging_hour,utilization
430757,VinodolHEP HE,2021-05-10 12:00:00+00:00,31.666667
430758,VinodolHEP HE,2021-05-10 13:00:00+00:00,98.333333
430759,VinodolHEP HE,2021-05-10 14:00:00+00:00,93.333333
430760,VinodolHEP HE,2021-05-10 15:00:00+00:00,100.000000
430761,VinodolHEP HE,2021-05-10 16:00:00+00:00,100.000000
...,...,...,...
433397,VinodolHEP HE,2022-10-24 15:00:00+00:00,3.333333
433398,VinodolHEP HE,2022-10-26 07:00:00+00:00,98.333333
433399,VinodolHEP HE,2022-10-26 08:00:00+00:00,45.000000
433400,VinodolHEP HE,2022-10-28 13:00:00+00:00,70.000000


In [61]:
dfv.apply(lambda x: x['charging_hour'].date(), axis=1)
dfv

,location_friendlyname,charging_hour,utilization
430757,VinodolHEP HE,2021-05-10 12:00:00+00:00,31.666667
430758,VinodolHEP HE,2021-05-10 13:00:00+00:00,98.333333
430759,VinodolHEP HE,2021-05-10 14:00:00+00:00,93.333333
430760,VinodolHEP HE,2021-05-10 15:00:00+00:00,100.000000
430761,VinodolHEP HE,2021-05-10 16:00:00+00:00,100.000000
...,...,...,...
433397,VinodolHEP HE,2022-10-24 15:00:00+00:00,3.333333
433398,VinodolHEP HE,2022-10-26 07:00:00+00:00,98.333333
433399,VinodolHEP HE,2022-10-26 08:00:00+00:00,45.000000
433400,VinodolHEP HE,2022-10-28 13:00:00+00:00,70.000000


Run Prophet and Test it using various settings, and on the various KPIs such as 

Report the results for each configuration.

